In [1]:
from main import main, training_loop


In [ ]:
for model in ["basic", "in_series", "cnn_lstm"]:
    training_loop(model)

('lemmas',)


In [7]:
from pathlib import Path
import joblib
import pandas as pd
res = {}
res_path = Path("./results/")
for _dir in res_path.iterdir():
    res[_dir.name] = {}
    for file in _dir.iterdir():
        one_res = joblib.load(file)
        res[_dir.name][file.name] = max(one_res.best_trial.intermediate_values.values())
df = pd.DataFrame.from_dict(res)
df.max(axis=1)

lemmas+no_stopwords.pkl          0.795620
clean+lemmas+no_stopwords.pkl    0.789386
clean+no_stopwords.pkl           0.787713
no_stopwords.pkl                 0.793035
clean.pkl                        0.789386
lemmas.pkl                       0.789386
clean+lemmas.pkl                 0.795772
dtype: float64